In [1]:
import pandas as pd
from openbb_terminal.api import openbb
from company import Company

from IPython.display import display


WARNING! Breaking changes incoming! Especially avoid using kwargs, since some of them will change.
For more information see the official documentation at: https://openbb-finance.github.io/OpenBBTerminal/api/


In [2]:
peers = [('VEEV', 1)] #, ('SPB', 2), ('SPCE', 3), ('ETSY', 4)]

for company in peers:
    
    stock = company[0]
    slot = company[1]

############## *** setting variables that hold dataframes: *** ###################
    df_balance = openbb.stocks.fa.yf_financials(stock, "balance-sheet") 
    df_data = openbb.stocks.fa.data(stock) # keys called later: 'Profit Margin', 'Insider Own', 'Insider Trans', 'Short Float', 'Inst Own', 'Inst Trans', 'Dividend'
    df_divs = openbb.stocks.fa.divs(stock)
    df_income = openbb.stocks.fa.fmp_income(stock) # keys called later: 'Gross profit ratio'
    
    
    df_metrics = openbb.stocks.fa.fmp_metrics(stock) # keys called later: 'Book value per share', 'Ptb ratio', 'Pe ratio', 
                                                     # 'Pfcf ratio', 'Dividend yield', 'Roe', 'Return on tangible assets', 
                                                     # 'Current ratio', 'Debt to equity', 'Price to sales ratio'
                                                                           
    # fmp api will throw error and return an empty list instead of requested dataframe      
    if not isinstance(df_metrics, pd.DataFrame):
        raise TypeError('Dag nabbit!' + ' ' + stock + ' is not available via FMP api. Failed to build df_metrics')
        
    # Where possible, converting strings to floats in df_metrics    
    float_error_set = set() 
    for column in df_metrics.columns:
        
        for row in df_metrics.index:
            
            try:
                df_metrics[column][row] = float(df_metrics[column][row])
                
            except ValueError: # catching the row name of any row that cannot be converted to float
                float_error_set.add(row)
                
    df_metrics_discarded = pd.DataFrame() # will hold the data we're removing from df_metrics in case we want it later
    for row in float_error_set: # Adding appropriate rows to the new dataframe
        df_metrics_discarded[row] = df_metrics.loc[row]

    df_metrics_discarded = df_metrics_discarded.T # transposing so it matches format of df_metrics
    
    # Removing any row from df_metrics where values cannot be converted to float
    df_metrics = df_metrics.drop(index= float_error_set)
    
    # Creating new column that will hold the average of each row
    df_metrics['5yr Avg'] = df_metrics.mean(axis=1)
                
    df_profile = openbb.stocks.fa.profile(stock) # keys called later: 'companyName', 'sector', 'industry' 
    df_quote = openbb.stocks.fa.quote(stock) # keys called later: 'Price', 'Market cap'

############## *** setting variables that hold raw data sequences: *** ###########

    # Getting Stock Twits sentiment data as a tuple:
    tuple_twits = openbb.stocks.ba.bullbear(stock)

##################################################################################    
############## *** setting variables that we'll add to company objects: *** ######  
##################################################################################         
    
############## *** BASIC DETAILS *** ##############

    name = df_profile.loc['companyName'][0]
    ticker = stock
    sector = df_profile.loc['sector'][0]
    industry = df_profile.loc['industry'][0]
    cap = df_quote.loc['Market cap'][0]
    price = df_quote.loc['Price'][0]
    
    # Series will be added to company object
    s_basic = pd.Series({'name': name, 'ticker': ticker, 'sector': sector, 'industry': industry, 'cap': cap, 'price': price})
    
############## *** VALUE METRICS *** ##############

    try: # If company has no long term debt 0 will be assigned to associated variable
        tca = df_balance.loc['Total current assets'][0]
        tld = df_balance.loc['Long-term debt'][0]
        
    except KeyError:
        tca = df_balance.loc['Total current assets'][0]
        tld = 0     

    tca_m_tld = tca - tld    

    ptb_mrq = df_data.loc['P/B'][0]
    ptb_ttm = df_metrics.loc['Ptb ratio'][0]
    ptb_5yr_avg = df_metrics.loc['Ptb ratio']['5yr Avg']

    bvps_mrq = df_data.loc['Book/sh'][0]
    bvps_ttm = df_metrics.loc['Book value per share'][0]
    bvps_5yr_avg = df_metrics.loc['Book value per share']['5yr Avg']
    
    try: # Because some companies do not have earnings
        pe_mrq = df_data.loc['P/E'][0]
        pe_ttm = df_metrics.loc['Pe ratio'][0]
        pe_5yr_avg = df_metrics.loc['Pe ratio']['5yr Avg']
        
    except KeyError:
        pe_mrq = 'n/a'
        pe_ttm = 'n/a'
        pe_5yr_avg = 'n/a'
        
    try: 
        pfcf_mrq = df_data.loc['P/FCF'][0]
        pfcf_ttm = df_metrics.loc['Pfcf ratio'][0]
        pfcf_5yr_avg = df_metrics.loc['Pfcf ratio']['5yr Avg']
        
    except KeyError:
        pfcf_mrq = 'n/a'
        pfcf_ttm = 'n/a'
        pfcf_5yr_avg = 'n/a'
    
    try: # Because some companies do not have sales
        pts_mrq = df_data.loc['P/S'][0]
        pts_ttm = df_metrics.loc['Price to sales ratio'][0]
        pts_5yr_avg = df_metrics.loc['Price to sales ratio']['5yr Avg']
        
    except KeyError:
        pts_mrq = 'n/a'
        pts_ttm = 'n/a'
        pts_5r_avg = 'n/a'

    # Series will be added to company object    
    s_value = pd.Series({'tca': tca, 'tld': tld, 'tca_m_tld': tca_m_tld, 'ptb_mrq': ptb_mrq, 'ptb_ttm': ptb_ttm, 'ptb_5yr_avg': ptb_5yr_avg,
                         'bvps_mrq': bvps_mrq, 'bvps_ttm': bvps_ttm, 'bvps_5yr_avg': bvps_5yr_avg, 'pe_mrq': pe_mrq, 'pe_ttm': pe_ttm, 'pe_5yr_avg': pe_5yr_avg,
                         'pfcf_mrq': pfcf_mrq, 'pfcf_ttm': pfcf_ttm, 'pfcf_5yr_avg': pfcf_5yr_avg, 'pts_mrq': pts_mrq, 'pts_ttm': pts_ttm, 'pts_5yr_avg': pts_5yr_avg})
        
############## *** MANAGEMENT METRICS *** ##############

    roe = df_metrics.loc['Roe'][0]
    roa = df_metrics.loc['Return on tangible assets'][0]
    gpr = df_income.loc['Gross profit ratio'][0]
    
    try: # Some companies are do not have profit margin
        pm = df_data.loc['Profit Margin'][0]
        
    except KeyError:
        pm = '0%'

    cr = df_metrics.loc['Current ratio'][0]
    dte = df_metrics.loc['Debt to equity'][0]
        
############## *** INSIDER DATA *** ##############

    io = df_data.loc['Insider Own'][0]
    it = df_data.loc['Insider Trans'][0]
    
    try:
        inst_o = df_data.loc['Inst Own'][0]
        
    except KeyError:
        inst_o = 'n/a'
     
    try:
        inst_t = df_data.loc['Inst Trans'][0]
        
    except KeyError:
        inst_o = 'n/a'
    
############## *** DIVIDEND DATA *** ##############

    try: # If company provides no dividend 0 will be assigned
        div = df_data.loc['Dividend'][0]
    
    except KeyError:
        div = 0
   
    try: # If company provides no dividend 0 will be assigned
        div_y = df_metrics.loc['Dividend yield'][0]
        
    except KeyError:
        div_y = 0
        
    div_h = df_divs
    
############## *** PUBLIC SENTIMENT & SHORT INTEREST DATA *** ##############
    
    if tuple_twits[1] != 0 and tuple_twits[2] != 0: # Normal case
        twits_perc = tuple_twits[2] / tuple_twits[1]
        
    elif tuple_twits[1] == 0: # Case for if there are no ratings
        twits_perc = 'Not Rated'
        
    elif tuple_twits[2] == 0: # Case for if all ratings are bearish
        twits_perc = 0
        
    shrt_int = df_data.loc['Short Float'][0]
        
############## *** COMPANY, SECTOR, INDUSTRY NEWS & SENTIMENT *** ##############

    # using df_profile to get companyName used for search string
    df_company_news = openbb.common.news(df_profile.loc['companyName', 0], sort='published')
    df_company_news = df_company_news.head(20)        
    
    # using df_profile to get sector used for search string
    df_sector_news = openbb.common.news(df_profile.loc['sector', 0] + 'Sector News Stock Market', sort='published')
    df_sector_news = df_sector_news.head(50)
    
    # using df_profile to get industry used for search string
    df_industry_news = openbb.common.news(df_profile.loc['industry', 0] + 'Industry News Stock Market', sort='published')
    df_industry_news = df_industry_news.head(50)  

    # Getting sentiment from last 10 days of news articles
    df_news_sent = openbb.stocks.ba.headlines(stock)
    sent_list = []

    for i in df_news_sent.index:
        sent_list.append(float(df_news_sent.loc[i][0]))
        
    news_sent = sum(sent_list)/len(sent_list)

############## *** ANALYST RATINGS *** ##############

    # creating ratings list which contains analyst ratings from last 10 days
    df_rating = openbb.stocks.dd.rating(stock)
    df_rating = df_rating.T
    df_rating = df_rating.loc['Rating']

    ratings_list = []
    for i in range(0, 10):
        ratings_list.append(df_rating[i])

    # Analyst recommendation totals by type over last 3 months
    df_rot = openbb.stocks.dd.rot(stock)
    df_rot = df_rot.T
    df_rot = df_rot[0]
    rot_dict = {'Strong Buy': df_rot['strongBuy'], 'Buy': df_rot['buy'], 'Hold': df_rot['hold'], 'Sell': df_rot['sell'], 'Strong Sell': df_rot['strongSell']}

    # Warren Buffet Investing Score
    wb_score = openbb.stocks.fa.score(stock)
    if wb_score is None:
        wb_score = 'n/a'       



Fetching data. Please be patient

Fetching data. Please be patient

Fetching data. Please be patient

Ticker should be a NASDAQ 100 ticker or SP 500 ticker

In [3]:
s_value

tca             3107183000.0
tld                        0
tca_m_tld       3107183000.0
ptb_mrq                 7.54
ptb_ttm               11.934
ptb_5yr_avg          13.9042
bvps_mrq               20.94
bvps_ttm              19.325
bvps_5yr_avg         12.0858
pe_mrq                 57.05
pe_ttm                81.299
pe_5yr_avg           82.6532
pfcf_mrq               34.12
pfcf_ttm              45.452
pfcf_5yr_avg         55.3318
pts_mrq                12.64
pts_ttm               18.774
pts_5yr_avg          20.5078
dtype: object

In [ ]:
##################################################################################    
############## *** Creating Company objects: *** #################################  
##################################################################################   
    if slot == 1:

        company1 = Company(name, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc, shrt_int,
                           df_company_news, df_sector_news, df_industry_news, news_sent,
                           ratings_list, rot_dict, wb_score)

    elif slot == 2:

        company2 = Company(name, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc, shrt_int,
                           df_company_news, df_sector_news, df_industry_news, news_sent,
                           ratings_list, rot_dict, wb_score)
        
    elif slot == 3:

        company3 = Company(name, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc, shrt_int,
                           df_company_news, df_sector_news, df_industry_news, news_sent,
                           ratings_list, rot_dict, wb_score)
        
    elif slot == 4:

        company4 = Company(name, ticker, sector, industry, cap, price,
                           tca, tld, ptb, bvps, pe, pfcf, pts,
                           roe, roa, gpr, pm, cr, dte,
                           io, it, inst_o, inst_t,
                           div, div_y, div_h,
                           twits_perc, shrt_int,
                           df_company_news, df_sector_news, df_industry_news, news_sent,
                           ratings_list, rot_dict, wb_score)

In [ ]:
print(company1.fullname, '   ', company1.rot_dict)
print(company2.fullname, '   ', company2.rot_dict)
print(company3.fullname, '   ', company3.rot_dict)
print(company4.fullname, '   ', company4.rot_dict)

In [ ]:
for x in [ptb_mrq, ptb_ttm, ptb_5yr_avg]:
    print(x)

In [ ]:
df_metrics

In [ ]:
peer_list = openbb.stocks.sia.filter_stocks("United States", "Communication Services", "Internet Content & Information", "Large Cap")

In [ ]:
openbb.stocks.sia.stocks_data(peer_list, "roa")

In [ ]:
ptb

In [ ]:
display(df_data)

In [ ]:
def display_df(dataframes):
    for df in dataframes:
        display(df)
        print('\n')

def display_s(list_of_series):
    for series in list_of_series:
        display(series)
        print('\n')

def display_f(forumlaic_value):
    for value in forumlaic_value:
        print(str(value) + '\n')
    


In [ ]:
dataframes = [df_quote, df_data, df_metrics, df_balance, df_divs, df_profile] 

series = [s_income, s_rating, s_rot]

formulaic = [f_twits_percent]

In [ ]:
display_df(dataframes)
display_s(series)
display_f(formulaic)